<a href="https://colab.research.google.com/github/yoooopeeee/test/blob/master/QPARC4_exercise_qulacs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QPARC第4回演習 量子計算シミュレータQulacsの使用法

株式会社QunaSys  井辺洋平

本演習では量子コンピュータのシミュレータであるQulacsの使用法を、手を動かしながら学びます。  

次回の演習では量子アルゴリズムVQEの実装をQulacsを用いて行うため、しっかり身につけていきましょう。

## 【**重要**】テスト実行のお願い

QPARC第四回講義当日までに、以下のテスト用セルを実行（セルをクリックし、`Shift+Enter`）して、エラーなく実行できることを確認して下さい。

In [ ]:
# テスト用セル
! pip install qulacs
from qulacs import QuantumState
q = QuantumState(1)
print(q)

↑上記セルで、最終的に以下のような出力が得られれば成功です。開講をお待ち下さい。
```
 *** Quantum State ***
 * Qubit Count : 1
 * Dimension   : 2
 * State vector : 
(1,0)
(0,0)
```

#### 【うまくいかない場合】
以下の日程でプログラミング環境構築相談会を実施しますので、ご参加頂ければ幸いです。  
> ８月１９日　１５：００−１７：００  

Zoomリンク等詳細は追ってご連絡します。  
なおQPARCのTeams [トラブル対応](https://teams.microsoft.com/l/channel/19%3a683a1790f8a54dec9ae3979af26c9203%40thread.tacv2/%25E3%2583%2588%25E3%2583%25A9%25E3%2583%2596%25E3%2583%25AB%25E5%25AF%25BE%25E5%25BF%259C?groupId=40fb0f64-0cce-4bfa-ab99-98d5309fb44f&tenantId=95308d55-3759-463d-ab8b-177ab2dac310) チャンネルでも随時サポート致します。お気軽にご相談下さい。


## 目次

1. 量子状態の生成と初期化
2. 量子ゲート
3. 量子回路
4. オブザーバブル

## 【はじめに】シミュレータを用いる動機とQulacsの紹介

現在ある量子コンピュータ実機の計算能力の制限から、量子コンピュータ向けアルゴリズムの研究のためには、**シミュレータ**（量子コンピュータの動作を古典コンピュータで再現するソフトウェア）を用いるのが主流です。

本演習では、世界最高クラスの動作速度を持つ量子コンピュータのシミュレータ [**Qulacs**](https://github.com/qulacs/qulacs) （キュラックスと発音します）の使い方を説明します。

Qulacsの内部はC++で実装されており非常に高速で動作しますが、Pythonインタフェースを通して簡単に実装することができます。

<div align="center">
<img src="https://images.squarespace-cdn.com/content/v1/5bc8010ff4e531349f0454c0/1540818591348-DOHKN62RYQPIDPYGZEUK/ke17ZwdGBToddI8pDm48kEy6sb-oyAcWceekoMu2IIxZw-zPPgdn4jUwVcJE1ZvWhcwhEtWJXoshNdA9f1qD7UnCxNA8dHvmd7460Z7fbKF_eFT_NGfl3GiRF7BarUbHzvvVUjfygrHeTx7hXWgYig/qulacs_logo.png?format=100w" width=10%>
</div>

#### シミュレータのベンチマーク比較
参考までに、[IBM Qiskit](https://qiskit.org/)等他のシミュレータとのベンチマーク比較です。  
横軸はqubit数、縦軸は計算時間です。広範囲なqubit数の領域で**Qulacsが最も高速**です。  

<div align="center">
<img src="https://camo.githubusercontent.com/8102c73911d713074808f12e0e82efe44c147a0c/68747470733a2f2f73746f726167652e676f6f676c65617069732e636f6d2f71756e617379732f73696e676c657468726561645f706c6f74322e706e67" width=50%>
</div>

（注1）ベンチマーク計算の詳細は https://github.com/qulacs/qulacs#performance を御覧ください。  
（注2）[Google qsim](https://github.com/quantumlib/qsim)のみ浮動小数点演算に(`double`(64ビット)ではなく)`float`(32ビット)を用いているため単純比較はできません。  

## 1. 量子状態の生成と初期化

### 1-1. 量子状態の生成
以下のコードで量子状態 (`QuantumState`クラス) を生成できます。

In [ ]:
from qulacs import QuantumState

# 1-qubitの状態を生成
n = 1  # qubit数
state = QuantumState(n)

$n$が非常に大きい場合など、メモリが不足している場合は量子状態を生成できません。

### 1-2. 量子状態の詳細情報の取得
`QuantumState`クラスのオブジェクトをprintすると、量子状態の情報が出力されます。

生成した量子状態は $|0\rangle =\left(\begin{array}{c}
1
\\
0
\end{array}
\right)$ に初期化されています。

In [ ]:
print(state)

ここで、1行目の$(1, 0)$は $1 + 0 i$を、2行目は$0+0i$を表します。

※ `State vector`欄各行の第1成分、第2成分がそれぞれ実部、虚部に対応。

なお`get_vector`メソッドによっても状態ベクトルが得られます。ここで`j`は虚数単位です。

In [ ]:
print(state.get_vector())

#### 多量子ビットの場合

多量子ビットの場合も、qubit数`n`を変えることで同様に量子状態を生成できます。  
生成した量子状態は $|00\cdots0\rangle$ に初期化されています。

In [ ]:
# 2-qubitの状態を生成
n = 2  # qubit数
state = QuantumState(n)
print(state)

In [ ]:
print(state.get_vector())  # jは虚数単位

生成した量子状態は $|00\cdots0\rangle$ に初期化されています。

#### 【復習】状態ベクトルの成分表示

2量子ビットの状態を例に説明します。  
2量子ビットの状態は一般に

$$ |\psi \rangle = a_{00}|00\rangle + a_{01}|01\rangle + a_{10}|10\rangle + a_{11}|11\rangle$$

と表せます。

このときの係数（複素数）$a_{00}, a_{01}, a_{10}, a_{11}$を並べたものが状態ベクトルの成分表示です。

$$|\psi\rangle =\left(\begin{array}{c}
a_{00}
\\
a_{01}
\\
a_{10}
\\
a_{11}
\end{array}
\right)$$

Qulacsの`QuantumState`オブジェクトを`print`すると上記の成分表示が得られます。


例えば 状態が$ |\psi \rangle = |00\rangle$ の場合、

$$ |\psi \rangle = 1\ |00\rangle + 0\ |01\rangle + 0\ |10\rangle + 0\ |11\rangle  
=\left(\begin{array}{c}
1
\\
0
\\
0
\\
0
\end{array}
\right)$$

と表せるため、出力は以下の通りです。  

In [ ]:
# 2-qubitの状態を生成
n = 2  # qubit数
state = QuantumState(n)
print(state)

※ `State vector`欄各行の第1成分、第2成分がそれぞれ実部、虚部に対応。

※ 各行の第1成分、第2成分がそれぞれ実部、虚部に対応。

2量子ビット状態の成分表示について、以下の対応が成り立ちます。

$$|00\rangle =\left(\begin{array}{c}
1
\\
0
\\
0
\\
0
\end{array}
\right),  \
|01\rangle =\left(\begin{array}{c}
0
\\
1
\\
0
\\
0
\end{array}
\right),\
|10\rangle =\left(\begin{array}{c}
0
\\
0
\\
1
\\
0
\end{array}
\right),\
|11\rangle =\left(\begin{array}{c}
0
\\
0
\\
0
\\
1
\end{array}
\right)
$$

### 1-3. 量子状態の初期化

生成した量子状態は、二進数を用いて初期化(`set_computational_basis`)したり、ランダムな状態に初期化(`set_Haar_random_state`)することができます。

#### 計算基底への初期化

量子計算分野において、{$|0\rangle, |1\rangle$}を量子ビットの個数だけテンソル積を取ったものを **計算基底 (computational basis)** と呼ぶ。    
例えば2量子ビット系の場合、計算基底は {$|00\rangle, |01\rangle$, $|10\rangle, |11\rangle$} となる。

In [ ]:
# |01> に初期化
state.set_computational_basis(0b01)  # 0b01の「0b」は2進数表示を表す
print(state.get_vector()) 

#### ランダムな状態への初期化

In [ ]:
# ランダムな初期状態を生成
state.set_Haar_random_state()
print(state.get_vector())

#### 【問題】
1. 状態ベクトル$|101\rangle$を成分表示するとどうなるでしょうか？
2. 状態ベクトル$|101\rangle$をQulacsを用いて生成・成分表示し、予想と一致することを確かめて下さい

#### 【解答】
3 qubitの状態は一般に$2^3=8$次元の複素数ベクトル

$$|\psi\rangle =\left(\begin{array}{c}
a_{000}
\\
a_{001}
\\
a_{010}
\\
a_{011}
\\
a_{100}
\\
a_{101}
\\
a_{110}
\\
a_{111}
\end{array}
\right)$$

で表されます。  

したがって$|101\rangle$は$a_{101}$のみ1, 他は0のベクトルとなります。

※ 2進法の101=10進法の5なので、0から数えて上から5番目の成分のみ1という解釈もできます。

In [ ]:
state = QuantumState(3)
state.set_computational_basis(0b101)
print(state) 

### 1-4. 量子状態のデータのコピー
量子状態を複製(`copy`)したりできます。

In [ ]:
# 1つ目の量子状態
state = QuantumState(2)
state.set_computational_basis(0b01)
print(state.get_vector())

In [ ]:
# コピーして新たな量子状態を作成
second_state = state.copy()
print(second_state.get_vector())

## 2. 量子ゲート

### 2-1. 量子ゲートの生成と作用
デフォルトで実装されている量子ゲートは`qulacs.gate`モジュールで定義されます。  
このモジュールにより、様々なゲート操作を行うことが可能です。

In [ ]:
from qulacs import QuantumState
from qulacs.gate import X, RY, DenseMatrix

#### Xゲート

まず1量子ビット操作の内Xゲートを掛けてみます。
Xの行列表記は  

$$
X=
\left(\begin{array}{cc}
0 & 1
\\
1 & 0
\end{array}
\right)\;\;\;
$$

であり、以下のように量子ビットを反転させる働きをします。  

$$X|0\rangle = |1\rangle, \;\;
X|1\rangle = |0\rangle
$$

例として、3つの量子ビットの内真ん中のqubitのみにXゲートを作用させてみましょう。

ここで、Qulacsではqubitは右端から0番目, 1番目, ...と数えることに注意が必要です。すなわち

$$
| q_2 q_1 q_0\rangle
$$

と並んでいるため、「1番目」のqubit $q_1$のみを指定してXゲート演算をします。

In [ ]:
# 1st-qubitにX操作するゲートx_gateを作成
index = 1
x_gate = X(index)

In [ ]:
# 量子状態の生成
n = 3
state = QuantumState(n)
print(state.get_vector())  # |000> = (1,0,0,0,0,0,0,0)

`update_quantum_state`メソッドを用いてゲート操作を行います。

In [ ]:
x_gate.update_quantum_state(state)
print(state.get_vector())  # |010> = (0,0,1,0,0,0,0,0)

#### Y回転ゲート
Y回転ゲート$R_y(\theta)$ も1 qubitゲートの一つであり、[Bloch球](https://ja.wikipedia.org/wiki/%E3%83%96%E3%83%AD%E3%83%83%E3%83%9B%E7%90%83)上で$y$軸に関する回転を表します。

$$
R_y(\theta) = e^{i \frac{\theta}{2}Y} 
= \cos{\frac{\theta}{2}} I + i \sin{\frac{\theta}{2}} Y
=\left(\begin{array}{cc}
\cos{\frac{\theta}{2}} & \sin{\frac{\theta}{2}} 
\\
-\sin{\frac{\theta}{2}} & \cos{\frac{\theta}{2} }
\end{array}
\right)\;\;\;
$$

In [ ]:
# 1st-qubitをYパウリでpi/4.0回転
from math import pi
angle = pi / 4.0
ry_gate = RY(index, angle)
ry_gate.update_quantum_state(state)
print(state.get_vector())

このように、回転ゲート<code>RX</code>,<code>RY</code>,<code>RZ</code>,<code>PauliRotation</code>は所定のパウリ演算子$P$、引数$\theta$に対して$\exp(i\frac{\theta}{2}P)$という操作を行います。それぞれのゲートの詳細は[APIドキュメント](http://qulacs.org/namespacegate.html)をご参照ください。


なお、Qulacs上で定義される回転ゲートは、Nielsen-Chuangのテキスト等で定義される$R_y(\theta)$とは、回転角の**符号が異なる**ので注意が必要です。

#### その他のゲート
Qulacs上で定義されているゲートは以下の通りです。

- single-qubit Pauli operation: Identity, X,Y,Z
- single-qubit Clifford operation : H,S,Sdag, T,Tdag,sqrtX,sqrtXdag,sqrtY,sqrtYdag
- two-qubit Clifford operation : CNOT, CZ, SWAP
- single-qubit Pauli rotation : RX, RY, RZ
- General Pauli operation : Pauli, PauliRotation
- IBMQ basis-gate : U1, U2, U3
- General gate : DenseMatrix
- Measurement : Measurement
- Noise : BitFlipNoise, DephasingNoise, IndepenedentXZNoise, DepolarizingNoise

### 2-2. 量子ゲートの合成
続けて作用する量子ゲートを合成し、新たな単一の量子ゲートを生成できます。

In [ ]:
from qulacs import QuantumState
from qulacs.gate import X, RY, merge

In [ ]:
# Xゲート作成
index = 0
x_gate = X(index)

# RYゲート（y軸に関する回転）作成
from math import pi
angle = pi / 2.0
ry_gate = RY(index, angle)

2つのゲートを合成して
$$
R_{y}(\theta) X
$$
を作成し、状態ベクトル$|0\rangle$に作用させます。

In [ ]:
# 量子状態の初期化
n = 1
state = QuantumState(n)

# ゲートを合成して新たなゲートを生成
x_and_ry_gate = merge(x_gate, ry_gate)  # 第一引数が先に作用する
x_and_ry_gate.update_quantum_state(state)
print(state.get_vector())

#### 【問題】

1. 2つのゲートの順序を入れ替えた $X R_{y}(\theta) |0\rangle $ を計算し、もとの $R_y (\theta) X|0\rangle$と比較して下さい。
2. 更に角度$\theta$の符号を反転させた $X R_{y}(-\theta) |0\rangle $ を計算して下さい。

#### 【解答】

以下の通りです。  
問題1の結果はもとの状態と異なりますが、問題2の結果はもとの状態と一致します。

実は、一般に $R_{y}(\theta) X= X R_y (-\theta)$が成り立ちます（交換関係$YX = -XY$の帰結）。

In [ ]:
# 問題1
state = QuantumState(1)

# ゲートを合成して新たなゲートを生成
ry_and_x_gate = merge(ry_gate, x_gate)  # 第一引数が先に作用する
ry_and_x_gate.update_quantum_state(state)
print(state.get_vector())

In [ ]:
# 問題2
state = QuantumState(1)

# 回転角度θを逆にしたRYゲート
ry_gate_inv = RY(index, -angle)

# ゲートを合成して新たなゲートを生成
ry_and_x_gate = merge(ry_gate_inv, x_gate)  # 第一引数が先に作用する
ry_and_x_gate.update_quantum_state(state)
print(state.get_vector())

### 2-3. 一般的な量子ゲートの実現
`qulacs.gate.DenseMatrix`を使うと、一般の行列からゲートを生成することができます。  
なお、Qulacs上で既に定義されているゲート（`H`, `RY`, `CNOT`等）については、それらを用いた方が`DenseMatrix`よりも高速です。

In [ ]:
from qulacs.gate import DenseMatrix
n = 2
state = QuantumState(n)
print(state)

In [ ]:
# 1-qubit gateの場合
gate = DenseMatrix(1, [[0,1],[1,0]])  # 1st-qubitに Xゲートを作用
print(gate)

In [ ]:
# 1st-qubitにゲート行列で作成したゲートを作用
# (qubitの順番は右端から0, 1,...と数えることに注意)
gate.update_quantum_state(state)  # |00>を|10>に 
print(state.get_vector())

In [ ]:
# 2-qubit gateの場合
gate = DenseMatrix([0,1], [[1,0,0,0],[0,1,0,0],[0,0,0,1],[0,0,1,0]])  # CNOTゲート cf. https://dojo.qulacs.org/ja/latest/notebooks/1.3_multiqubit_representation_and_operations.html
print(gate)

In [ ]:
# CNOTゲートを作用
gate.update_quantum_state(state)  # |10>を|11>に
print(state.get_vector())

## 3. 量子回路

### 3-1. 量子回路の構成
量子コンピュータでは一般に多くの量子ゲートを決まった順番で作用させますが、量子回路としてそれらをまとめておくことでより扱いやすくなります。
例えばVQEなど、同じ量子回路を繰り返し作用させる場合特に便利です。  
量子回路(`QuantumCircuit`クラス)は量子ゲートの集合として表され、以下のように量子回路を構成できます。  

In [ ]:
from qulacs import QuantumState, QuantumCircuit
from qulacs.gate import Z
n = 3
state = QuantumState(n)
state.set_zero_state()

# 量子回路を定義
circuit = QuantumCircuit(n)

# 量子回路にhadamardゲートを追加
for i in range(n):
    circuit.add_H_gate(i)

# ゲートを生成し、それを追加することもできる。
for i in range(n):
    z_gate = Z(i)
    circuit.add_gate(z_gate)

# 量子回路を状態に作用
circuit.update_quantum_state(state)

print(state.get_vector())

## 4. オブザーバブル

量子力学においては、**物理量はエルミート演算子**$A$**で表され、オブザーバブルとも呼ばれます**。  
（エルミート演算子は$A^\dagger=A$を満たす演算子）  


状態$|\psi\rangle$について$A$の「**射影測定**」というものを行うと、$|\psi\rangle$をAの固有状態$\{|a_i\rangle\}_i$（固有値$a_i$）で展開した係数

$$
 |\psi\rangle = \sum_i c_i |a_i\rangle
$$

に応じた確率$|c_i|^2$で測定値$a_i$が得られ、その期待値は$\langle\psi|A|\psi\rangle$となります。



### 4-1. オブザーバブルの生成
Qulacsでは、オブザーバブルはパウリ演算子$X,Y,Z$の直積（一般化パウリ行列）として表現されます。  
（エルミートな演算子は必ずパウリ演算子の直積の和で表されるため。）  
パウリ演算子は下記のように定義できます。

以下では $2 X_0 X_1 Y_2$ というオブザーバブルを定義しています。  

In [ ]:
from qulacs import Observable
n = 3
coef = 2.0
# 2.0 X_0 X_1 Y_2 というパウリ演算子を設定
Pauli_string = "X 0 X 1 Y 2"
observable = Observable(n)
observable.add_operator(coef,Pauli_string)

一般のオブザーバブル（エルミート演算子）は、上記の要領で、パウリ演算子の直積（一般化パウリ行列）に実係数をかけたものを`add_operator`メソッドにより足していくことで構成できます。

### 4-2. オブザーバブルの評価
`get_expectation_value`メソッドにより、状態に対してオブザーバブルの期待値を評価できます。

In [ ]:
state = QuantumState(n)
state.set_Haar_random_state()

# 期待値の計算
value = observable.get_expectation_value(state)
print(value)

#### 【問題】
以下のランダムに生成した1量子ビット状態`state`について、オブザーバブル$X$の期待値を、以下2通りの方法で求めて下さい。
1. $X$に対応するオブザーバブルを定義し期待値を直接求める
2. $Z$に対応するオブザーバブルを定義して求める（ヒント：$X = H^{\dagger} Z H$）

（余裕がある方はオブザーバブル$Y$についても同様に2通りの方法で計算してみて下さい。）

In [ ]:
state = QuantumState(1)
state.set_Haar_random_state()
print(state)

#### 【解答】
問題1.の解答は以下の通りです。

In [ ]:
# 問題1.

# observable Xの作成
coef = 1.
pauli_string = "X 0"
obs_x = Observable(1)
obs_x.add_operator(coef, pauli_string)

# Xに関する期待値測定
value = obs_x.get_expectation_value(state)
print(value)

問題2.の解答です。  
QPARC第3回講義で学んだ通り、量子コンピュータで基本的に可能な測定は0, 1ビットに関する**射影測定**、すなわち、$Z$の期待値のみを知ることができます。  
ある状態の$X$の期待値$\langle \psi |X |\psi\rangle$を知りたい場合、関係式
$$ \langle \psi |X |\psi\rangle=  \langle \psi | H^{\dagger} Z H |\psi\rangle$$
を利用します。  
すなわち、状態$|\psi\rangle$にまずアダマールゲート$H$を作用させ、得られた状態$H|\psi\rangle$について$Z$の期待値を測定することで、$\langle \psi |X |\psi\rangle$が得られます。

In [ ]:
# 問題2.

# observable Zの作成
coef = 1.
pauli_string = "Z 0"
obs_z = Observable(1)
obs_z.add_operator(coef, pauli_string)

# 状態にアダマールゲートHをかける
from qulacs.gate import H
hadamard = H(0)
hadamard.update_quantum_state(state)

# Zに関する期待値測定
value = obs_z.get_expectation_value(state)
print(value)

## おわりに

以上、大変駆け足ではありましたが、量子コンピュータ シミュレータの基本的な使い方から、分子のエネルギー計算という具体的なアルゴリズムの実装までを概観しました。

本チュートリアルで体験頂いたように、シミュレータの利用を通して量子コンピュータで計算する感覚を養うことで、現在より大規模かつ高精度の実機が登場した際にも円滑に対応できるはずです。

量子コンピュータ アルゴリズムに関して、さらに深く勉強されたい方は、Google Colabで実行しながら学べる無料教材[Quantum Native Dojo](https://dojo.qulacs.org/ja/latest/) をお勧めします。